# Conformational Swap Moves in Protein-Surface Interactions

_M. Lund, 2017_

In this Notebook we perform Metropolis MC simulations of peptides in contact with a planer surface.
Peptide flexibility is enabled by MC swap moves of conformations obtained from all-atom MD simulations, saved to a trajectory file.

The Notebook is at the moment very simple and uses short trajectory (converted from `xtc` to `pqr` format using VMD). For inspiration about Faunus notebooks, see http://github.com/mlund/faunus-notebooks.

Current fratures:

- arbitrary number of proteins
- internal degrees of freedom from external trajectory (see `traj` section in molecule list
- planar surface on one side of box (Gouy-Chapman and short range attraction)
- explicit surface sites (added in molecule list)
- possibility to use springs for internal degrees of freedom (see `fasta` section in molecule list)
- XTC trajectory and PQR file output for visualization in VMD

## System requirements

This Jupyter Notebook was originally run on MacOS 12.10 / Ubuntu 16.04 using Continuum Analytics Anaconda. The Anaconda environment can be generated using the following commands, assuming Miniconda has already been installed,

```.bash
    conda create --name SIenv python=3 matplotlib=1.5.3 scipy=0.18.1 pandas=0.19.1 notebook=4.2.3
    source activate SIenv
```

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np, pandas as pd
import os.path, os, sys, json, shutil
from pathlib import Path
import math
from math import sqrt, log, pi, exp, sinh
plt.rcParams.update({'font.size': 16, 'figure.figsize': [8.0, 6.0]})
try:
    workdir
except NameError:
    workdir=%pwd
else:
    %cd -q $workdir
print(workdir)

/home/mikael/github/configurationswap


## Download and build MC software _Faunus_

The main program is `fasta.cpp` which we crudely copy into the main faunus directory so that no new build taget needs to be set up.

In [2]:
%%bash -s "$workdir"
cd $1

if [ ! -d "faunus" ]; then
  git clone https://github.com/mlund/faunus.git
  cd faunus
  git checkout aa6f933ffa4b496c08a9994554de5d30971db8f7
else
  cd faunus
fi

# if different, copy custom cpp into faunus
if ! cmp ../mc/fasta.cpp src/examples/twobody.cpp >/dev/null 2>&1
then
    cp ../mc/fasta.cpp src/examples/twobody.cpp
fi

cmake . -DCMAKE_BUILD_TYPE=RelWithDebInfo &>/dev/null
make example_twobody -j4
cd $1

[ 33%] Built target xdrfile
[ 83%] Built target libfaunus
[100%] Built target example_twobody


In [3]:
def mkinput():
    j = {
      "atomlist" : {
        "Na"   :  { "q": 1, "r":1.9, "dp": dp, "mw":22.99 },
        "SiO"  :  { "q":-1, "r":0.0, "dp": dp, "mw":1 },
#       "SiOH" :  { "q":-1, "r":0.0, "dp": dp, "mw":1 },
        "ASP"  :  { "q":-1, "r":3.1, "dp": dp, "mw":110 },
        "HASP" :  { "q":0,  "r":3.1, "dp": dp, "mw":110 },
        "LASP" :  { "q":2,  "r":3.6, "dp": dp, "mw":110 },
        "CTR"  :  { "q":-1, "r":1.6, "dp": dp, "mw":16 },
        "HCTR" :  { "q":0,  "r":1.6, "dp": dp, "mw":16 },
        "GLU"  :  { "q":-1, "r":3.2, "dp": dp, "mw":122 },
        "HGLU" :  { "q":0,  "r":3.2, "dp": dp, "mw":122 },
        "LGLU" :  { "q":2,  "r":3.8, "dp": dp, "mw":122 },
        "HIS"  :  { "q":0,  "r":3.3, "dp": dp, "mw":130 },
        "HHIS" :  { "q":1,  "r":3.3, "dp": dp, "mw":130 },
        "NTR"  :  { "q":0,  "r":1.5, "dp": dp, "mw":14 },
        "HNTR" :  { "q":1,  "r":1.5, "dp": dp, "mw":14 },
        "TYR"  :  { "q":-1, "r":3.5, "dp": dp, "mw":154 },
        "HTYR" :  { "q":0,  "r":3.5, "dp": dp, "mw":154 },
        "LYS"  :  { "q":0,  "r":3.2, "dp": dp, "mw":116 },
        "HLYS" :  { "q":1,  "r":3.2, "dp": dp, "mw":116 },
        "CYS"  :  { "q":-1, "r":3.6, "dp": dp, "mw":103 },
        "HCYS" :  { "q":0,  "r":3.6, "dp": dp, "mw":103 },
        "ARG"  :  { "q":0,  "r":3.4, "dp": dp, "mw":144 },
        "HARG" :  { "q":1,  "r":3.4, "dp": dp, "mw":144 },
        "ALA"  :  { "q":0,  "r":2.6, "dp": dp, "mw":66 },
        "ILE"  :  { "q":0,  "r":3.6, "dp": dp, "mw":102 },
        "LEU"  :  { "q":0,  "r":3.6, "dp": dp, "mw":102 },
        "MET"  :  { "q":0,  "r":3.8, "dp": dp, "mw":122 },
        "PHE"  :  { "q":0,  "r":3.4, "dp": dp, "mw":138 },
        "PRO"  :  { "q":0,  "r":3.4, "dp": dp, "mw":90 },
        "TRP"  :  { "q":0,  "r":4.3, "dp": dp, "mw":176 },
        "VAL"  :  { "q":0,  "r":3.4, "dp": dp, "mw":90 },
        "SER"  :  { "q":0,  "r":2.8, "dp": dp, "mw":82 },
        "THR"  :  { "q":0,  "r":3.5, "dp": dp, "mw":94 },
        "ASN"  :  { "q":0,  "r":3.6, "dp": dp, "mw":108 },
        "GLN"  :  { "q":0,  "r":3.8, "dp": dp, "mw":120 },
        "GLY"  :  { "q":0,  "r":2.5, "dp": dp, "mw":54 }
        },

      "processes" : {
        "H-Asp" : { "bound":"HASP" , "free":"ASP" , "pKd":4.0  , "pX": pH },
        "H-Ctr" : { "bound":"HCTR" , "free":"CTR" , "pKd":3.8  , "pX": pH },
        "H-Glu" : { "bound":"HGLU" , "free":"GLU" , "pKd":4.4  , "pX": pH },
        "H-His" : { "bound":"HHIS" , "free":"HIS" , "pKd":6.3  , "pX": pH },
        "H-Arg" : { "bound":"HARG" , "free":"ARG" , "pKd":12.0 , "pX": pH },
        "H-Ntr" : { "bound":"HNTR" , "free":"NTR" , "pKd":7.5  , "pX": pH },
        "H-Cys" : { "bound":"HCYS" , "free":"CYS" , "pKd":9.5  , "pX": pH },
        "H-Tyr" : { "bound":"HTYR" , "free":"TYR" , "pKd":9.6  , "pX": pH },
        "H-Lys" : { "bound":"HLYS" , "free":"LYS" , "pKd":10.4 , "pX": pH }#,
#       "H-SiO" : { "bound":"SiOH" , "free":"SiO" , "pKd":8.0  , "pX": pH }
        },

      "energy" : {
          "gouychapman" : { "rho" : -7.8019e-3 },
          "nonbonded" : {
            "coulomb"  : { "epsr" : 78.54, "ionicstrength": salt },
            "ljsimple" : { "eps":0.05 },
            "harmonic" : { "k":0.76, "req":4.0 }
            }
          },
      "stickywall_type" : "lj",
      "stickywall_depth" : 0.9,

      "moves" : {
          "//atomtranslate"   : { "protein" : { "prob":1.0, "permol":True, "peratom":True } },
          "moltransrot"      : { "protein" : { "dp":5, "dprot":1, "permol":True }  },
          "conformationswap" : { "protein" : { "permol":True }  },
          "//crankshaft"      : { "protein" : { "dp":6, "maxlen":6, "permol":True } },
          "//pivot"           : { "protein" : { "dp":6, "maxlen":6, "permol":True } },
          "//titrate"         : { "prob":0.2 }
          },

      "moleculelist" : {
          "surface" : { "Ninit":1, "structure":"surface.aam", "insdir":"0 0 0", "insoffset":"0 0 " + str(boxlen/2-0.001), "checkoverlap":False, "rotate":False},
          "fastaprotein" : { "Ninit":0, "fasta":"DSHAKRHHGYKRKFHEKHHSHRGY"}, # histatin 5
          "protein" : { "Ninit":3, "traj":"cg_ter_traj.pqr", "centertraj":True} # histatin 5
          },

      "system" : {
          "temperature"  : 298,
          "cuboid"       : { "xyzlen" : "150 150 "+str(boxlen) },
          "mcloop"       : { "macro":10, "micro": micro }
          },
      "analysis" : {
          "polymershape" : { "nstep":20, "mollist":[ "protein" ] },
          "xtcfile" :   { "file": "traj.xtc", "nstep":50 },
          "pqrfile" :   { "file": "confout.pqr" },
          "statefile" : { "file": "state" }
          }
      }
    with open('fasta.json', 'w+') as f:
        f.write(json.dumps(j, indent=4))

In [5]:
%cd $workdir/mc

boxlen = 300 # box length (angstrom)
salt=0.1 # (mol/l)
pH=7
dp=10
micro=1000

mkinput()
!../faunus/src/examples/twobody

/home/mikael/github/configurationswap/mc
Centering conformations in trajectory file cg_ter_traj.pqr. Done.
Writing to file 'init.pqr'. OK!
# Reading process H-Arg ... OK!
# Reading process H-Asp ... OK!
# Reading process H-Ctr ... OK!
# Reading process H-Cys ... OK!
# Reading process H-Glu ... OK!
# Reading process H-His ... OK!
# Reading process H-Lys ... OK!
# Reading process H-Ntr ... OK!
# Reading process H-Tyr ... OK!
Reading space state file 'state'. OK!
  Read 82 particle(s).
  Read 4 group(s).
  Restoring random number generator state.

 ...............................
  Simulation Space and Geometry  
 *******************************
  Boundary                 Cuboid (XY-periodicity)
  Volume                   6.75e+06 Å³ = 6750 nm³ = 6.75e-21 liters
  Sidelengths              150 150 300 (Å)
  Scale directions         XYZ
  Number of particles      82
  Electroneutrality        NO! 15
  System sanity check      Passed
  Number of molecule types 3
  Groups:
    1     [0-25]   

In [ ]:
!vmd confout.pqr traj.xtc